<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/LLM_fine_tune(RLHF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch trl pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00


In [2]:
import random

import numpy as np
import torch
import pandas as pd

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
)


def set_seed(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

output_dir = "supervised-summarize-checkpoint"
train_batch_size = 16
gradient_accumulation_steps = 1
learning_rate = 1e-5
eval_batch_size = 1
eval_steps = 500
max_input_length = 550
save_steps = 1000
num_train_epochs = 20
random.seed(42)


In [3]:
df = pd.read_parquet("/content/test-00000-of-00001-59ffb27399371eac.parquet")

In [4]:
df.iloc[12]

prompt    SUBREDDIT: r/tifu\nTITLE: TIFU bY brushing wit...
label     Brush Teeth with Baking Soda without research,...
Name: 12, dtype: object

In [5]:
import json

import pandas as pd
import torch
from datasets import load_dataset
from torch.utils.data import Dataset


class TLDRDataset(Dataset):
    def __init__(self, train_path, tokenizer, split, max_length=256):
        self.post_list = []
        dataset = pd.read_parquet(train_path)
        self.labels = []
#         dataset = dataset[:100]
        for sample in dataset.iterrows():
            self.post_list.append(sample[1]["prompt"])
            self.labels.append(sample[1]["label"])

        self.tokenizer = tokenizer
        self.max_length = max_length
        self.input_ids = []
        self.attn_masks = []

    def __len__(self):
        return len(self.post_list)

    def __getitem__(self, idx):
        txt = self.post_list[idx]
        label = self.labels[idx]

        encodings_dict = self.tokenizer(txt, truncation=True, max_length=self.max_length, padding="max_length")
        encodings_dict_label = self.tokenizer(label,truncation=True, max_length=self.max_length, padding="max_length")
        input_ids = torch.tensor(encodings_dict["input_ids"])
        attn_masks = torch.tensor(encodings_dict["attention_mask"])
        labels_ids = torch.tensor(encodings_dict_label["input_ids"])
        return {
            "input_ids": input_ids,
            "attention_mask": attn_masks,
            "labels":labels_ids,
        }


In [6]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/tiny_starcoder_py")
model = AutoModelForCausalLM.from_pretrained("bigcode/tiny_starcoder_py", use_cache=False).to("cuda:0")
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
# Set up the datasets
data_path = "/content/test-00000-of-00001-59ffb27399371eac.parquet"
train_dataset = TLDRDataset(
    data_path,
    tokenizer,
    "train",
    max_length=256,
)

In [8]:
for i in train_dataset:
    print(i["input_ids"], i["labels"])
    break


tensor([ 7100,   613,  2918,   780,    44,   540,    33, 40186,   203, 13777,
           44,  3110,   428,    35,    43,   506,    79,   623,  1672, 11970,
          428,    35,    43,   488,   614,   646,  3654,   415,   439,  1631,
         1159, 16661,  1246,  6366,   973,  3425,    32,   203,  3705,    44,
        12000, 17964,  3638,  1548,    32,   439,  9845,   458,  7735,  1330,
         5133, 31695,   432,   312,  7000,   372,  7660,   544,  2442,    30,
         1273,   439,  4763,  2583, 42289,   312,  3493,   963,   432,  1672,
         7713,  1412,   561, 12767,   372,   458, 18734,   308,    59,  4763,
         5054,  1755,  1591, 12112,  2670,    30,   461,   436,  5075, 17510,
           30,   561,  1597,   963,   432,   322, 48385,   547,   203,   203,
         7558,   395,    19,  2770,    30,   312, 17142,   432, 22599, 14818,
           30,   439,  7307, 29220,   372,   458,  3932,   107,   544, 18660,
           30,  3919,   312,  9525,  2350,   688,   996,  4528, 

In [9]:
torch.cuda.set_device(0)

In [10]:
import accelerate
from transformers import TrainingArguments, Trainer
# Prepare the trainer and start training
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=train_batch_size,
#     per_device_eval_batch_size=eval_batch_size,
    fp16=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=2,
    warmup_steps=100,
    logging_steps=10,
)


In [11]:
training_args.device.index

0

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
#     compute_metrics=compute_metrics,
#     data_collator=default_data_collator,
#     preprocess_logits_for_metrics=preprocess_logits_for_metrics
)
trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
10,10.188400
20,6.580500
30,2.587400
40,1.480900
50,1.194700
60,1.041800
70,1.044000
80,0.993800
90,1.014500
100,1.025400


In [13]:
trainer.save_model("summarization_policy_new/")

In [14]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("summarization_policy_new/")
model_path = "bigcode/tiny_starcoder_py"

tokenizer = AutoTokenizer.from_pretrained(model_path, truncation=True, max_length=256, padding="max_length")
text = df.iloc[2]["prompt"]
tokenized_text = tokenizer(text, return_tensors="pt", max_length=256)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [15]:
tokenizer.decode(model.generate(**tokenized_text)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 203, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


"SUBREDDIT: r/relationships\nTITLE: The girl [26 F] I [22 M] have been seeing for a month didn't respond to me at all yesterday while hanging out with a friend [~30? M].\nPOST: She gets terrible service while at her house, but I texted her 3 times yesterday, 4-5 hours apart. She didn't call me until early this morning and left a voicemail that she was busy all day with a friend who showed up out of the blue.\n\nI saw that she posted a picture of the two of them out of her dead zone house on facebook before I texted her the last time.\n\nI don't mind that she hangs out with friends, and I know it's pretty early in the relationship, but am I wrong to be a little annoyed that she didn't respond until 24 hours after my first text?\nTL;DR: <|endoftext|>"

#Step-2
##Building a reward Model

In [16]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments

In [17]:
MODEL_PATH = "bigcode/tiny_starcoder_py"
DATA_PATH = "/content/test_step2.parquet"

In [18]:
df = pd.read_parquet(DATA_PATH)
df = df[:10]
raw_dataset = Dataset.from_pandas(df)
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10
})

In [19]:
##defininig the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

In [20]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def formatting_func(examples):
    kwargs = {"padding": "max_length",
              "truncation": True,
              "max_length": 256,
              "return_tensors": "pt"
              }

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["prompt"] + "\n" + examples["chosen"]
    prompt_plus_rejected_response = examples["prompt"] + "\n" + examples["rejected"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

In [21]:
formatted_dataset = raw_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [22]:
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 7
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 3
    })
})

In [23]:
model.config


GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "validate_runner_input": tr

In [24]:
### Loading the TRL reward trainer and training the trainer
training_args = TrainingArguments(
        output_dir="rm_checkpoint/",
        num_train_epochs=2,
        logging_steps=10,
        gradient_accumulation_steps=1,
        save_strategy="steps",
        evaluation_strategy="steps",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=1,
        eval_accumulation_steps=1,
        eval_steps=500,
        save_steps=500,
        warmup_steps=100,
        logging_dir="./logs",
        learning_rate=1e-5,
        save_total_limit=1,
        no_cuda=True
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1281: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [25]:
trainer = RewardTrainer(model=model,
                        tokenizer=tokenizer,
                        train_dataset=formatted_dataset['train'],
                        eval_dataset=formatted_dataset['test'],
                        args= training_args
                        )
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:108: FutureWarning: Using `transformers.TrainingArguments` for `args` is deprecated and will be removed in a future version. Please use `RewardConfig` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:162: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:187: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` metho

Step,Training Loss,Validation Loss


TrainOutput(global_step=8, training_loss=0.7653201222419739, metrics={'train_runtime': 131.417, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.061, 'total_flos': 0.0, 'train_loss': 0.7653201222419739, 'epoch': 2.0})

In [26]:
trainer.save_model("rm_model/")

In [27]:
# Load the model from the local directory
rm_model = AutoModelForCausalLM.from_pretrained("rm_model/")

# Load the tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained("rm_model/")

In [28]:
def get_score(model, tokenizer, prompt, response):

    instructions = tokenizer.encode_plus(prompt,
                                       response,
                                       padding="max_length",
                                       max_length=256,
                                       return_tensors="pt",
                                        truncation=True)
    with torch.no_grad():
        outputs = model(**instructions)

    logits = outputs[0]

    return logits

In [29]:
# usage with prompt
prompt = df.iloc[0]["prompt"]
example_prefered_response = df.iloc[0]["chosen"]
example_unprefered_response = df.iloc[0]["rejected"]


In [30]:
loss1 = get_score(model, tokenizer, prompt, example_prefered_response)
loss2= get_score(model, tokenizer, prompt, example_unprefered_response)


In [31]:
from torch import nn
loss = -nn.functional.logsigmoid(loss1 - loss2).mean()

In [32]:
tokenizer.decode(torch.max(loss1, axis=-1).indices[0])

'_MEIT_\n       "r/#: r RelationshipR0]2ARCH\nlfriend\n2//M]\n me\n [\n thelf down the life\n a [ to\n\n: https [lfriend [ her don a aressed in my19 minutes.\n\n\'\'\'How updated:** girlfriend was through my friends**. I my my friends..** my  of lf.\n\n** was ading for my new personirl** I\'ve to findoolpping my my the future  but I was a in\n\n** have ali a of ago,,liting my my girl. and she had my about my.. my few of days.1viously). was\'t have that was).\n\n** was it with my lastirl and she had my Facebook, the   I wasand historyirl) was in March,\n to, be that were flied. I a gs. f.ing on her.\nirirl issY I1 girllfriend is 19 months. on my Facebook.. my permission. I it messages. my.lirting with me coupleirl.\n found up with me after she through further with She'

#STEP-3
##Building Policy Model

In [33]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

In [34]:
MODEL_PATH = "rm_model/"
DATA_PATH = "/content/test_step2.parquet"

In [35]:
df = pd.read_parquet(DATA_PATH)
df = df[:1000]
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [36]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name=MODEL_PATH, steps=51200, learning_rate=1.41e-5, remove_unused_columns=True
)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [37]:
from transformers import AutoTokenizer, pipeline

In [38]:
dataset = dataset.rename_columns({"prompt": "review"})
dataset = dataset.filter(lambda x: len(x["review"]) > 500, batched=False)
dataset = dataset.map(lambda x: {"review": x["review"][:1000]}, batched=False)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [39]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [40]:
txt_in_len = 5
txt_out_len = 32
seed = 1

dataset = dataset.map(
    lambda x: {"input_ids": tokenizer.encode(" " + x["chosen"], return_tensors="pt", truncation=True, padding="max_length", max_length=32)[0]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:20480]
from datasets import Dataset

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [41]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [42]:
rf_model_path = "rm_model/"
starcoder_model = AutoModelForCausalLMWithValueHead.from_pretrained("summarization_policy_new/")  ##policy model from step 1
starcoder_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path) ## reward model from step 2
starcoder_tokenizer = AutoTokenizer.from_pretrained("bigcode/tiny_starcoder_py") ## tokenizer of step 1 model., here since we are using same model for step 1 and 2 it doesnot matter
dataset

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})

In [43]:
import torch
optimizer = torch.optim.SGD(starcoder_model.parameters(), lr=config.learning_rate)
ppo_trainer = PPOTrainer(config, starcoder_model, starcoder_model, starcoder_tokenizer, dataset=dataset, data_collator=collator, optimizer=optimizer)


In [44]:
ctrl_str = ["[negative]", "[positive]"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
ctrl_tokens = dict((s, starcoder_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [45]:

def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == "[negative]":
            logit[i] = -logit[i]
        elif task[i] == "[positive]":
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

In [46]:
pos_logit_to_reward(torch.Tensor([4, 4]), ctrl_str)


tensor([-4.,  4.])

In [47]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": starcoder_tokenizer.eos_token_id,
    "max_new_tokens": 32,
    "eos_token_id": -1,
}

In [48]:
def get_score(model, tokenizer, responses):
    positive_logist = []
    for i in responses:
        instructions = tokenizer.encode_plus(
                                           i,
                                           padding="max_length",
                                           max_length=32,
                                           return_tensors="pt")
        with torch.no_grad():
            outputs = model(**instructions)

        logits = outputs[0].mean()
        positive_logist.append(logits)

    return positive_logist

In [57]:
from random import choices
from tqdm import tqdm
import time
import numpy as np

# Assuming device is defined (e.g., device = torch.device("cuda:0"))
# If not, you can assign a device based on your setup

for epoch in range(1):
    for batch in tqdm(ppo_trainer.dataloader):
        (logs, game_data,) = (
            dict(),
            dict(),
        )

        print(ctrl_str)
        #### prepend a random control token
        task_list = choices(ctrl_str, k=config.batch_size)

        game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]

        query_tensors = []
        for t, input_ids in zip(task_list, batch["input_ids"]):
            # Move ctrl_tokens[t] to the same device as input_ids
            ctrl_t_on_device = ctrl_tokens[t].to(input_ids.device)
            # Move input_ids to the same device as ctrl_tokens[t]
            query_tensors.append(torch.cat((ctrl_t_on_device, input_ids.to(ctrl_t_on_device.device))))

        #### get response from gpt2
        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(
                query,
                max_length=txt_out_len,  # Set max length directly
                num_return_sequences=1,  # Set the number of return sequences
                temperature=1.0,  # Set the temperature
                top_k=50,  # Set top_k directly
            )
            response_tensors.append(response.squeeze())

        game_data["response"] = [starcoder_tokenizer.decode(r.squeeze()) for r in response_tensors]

        #### sentiment analysis
        texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
        logits = get_score(starcoder_model, starcoder_tokenizer, texts)
        rewards = pos_logit_to_reward(logits, task_list)

        #### Run PPO training
        t = time.time()
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        for cs in ctrl_str:
            key = "env/reward_" + cs.strip("[]")
            stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
        ppo_trainer.log_stats(stats, game_data, rewards)


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 35, but `max_length` is set to 32. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


['[negative]', '[positive]']


  0%|          | 0/3 [01:09<?, ?it/s]


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
starcoder_model.save_pretrained("rhlfmodel/")
starcoder_tokenizer.save_pretrained("rhlfmodel/")


In [ ]:
from transformers import pipeline, set_seed
model_path = "rhlfmodel/"
set_seed(42)
pipe = pipeline("text-generation",model=model_path, tokenizer=model_path, max_length=30, num_return_sequences=5)

In [ ]:
text = dataset["rejected"][0]
pipe(text)